<a href="https://colab.research.google.com/github/groupby/Taxonomy/blob/master/Batch_rule_uploads_Module.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bulk Rule Curation

## Install and Imports needed to run the data

In [0]:
!pip install --upgrade -q pygsheets
!pip install tqdm
!pip install datetime

    100% |████████████████████████████████| 102kB 3.9MB/s 
    100% |████████████████████████████████| 61kB 2.5MB/s 
    100% |████████████████████████████████| 174kB 17.1MB/s 


In [0]:
gsheets_url = 'https://docs.google.com/spreadsheets/d/16fFSeMa31_K8UyHxzfXRWllBfzcn38l7Ks1TBb4VCTQ/edit#gid=1279399904'
sheet_name = 'MW Test'

In [0]:
import pandas as pd
import json
import datetime
import pygsheets
import requests
from tqdm import tqdm

import psycopg2

from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

## Establishing a connection to the Google Sheet

In [0]:
#getting the date to document the day of the upload
today = str(datetime.datetime.today())[:10]

In [0]:
#Connecting to google spreadsheets
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [0]:
spreadsheet = gc.open_by_url(gsheets_url)
wks = spreadsheet.worksheet(sheet_name)

In [0]:
#Converting the dictionary records to a pandas dataframe. 
df = pd.DataFrame(wks.get_all_records())

In [0]:
#the database credentials.
postgres = {
    "database":"feeddata",
    "user":"groupby",
    "password":"gbi123gbi",
    "host":"sandbox.edgecaseprod.com",
    "port":"5432"
}

In [0]:
# insert your JWT here.
JWT = """eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpZCI6NDE5LCJuYW1lIjoiUnlhbiBGcmlzZWxsYSIsImVtYWlsIjoicnlhbi5mcmlzZWxsYUBncm91cGJ5aW5jLmNvbSIsInBlcm1pc3Npb25zIjpbIkFjY2VzcyBEYXNoYm9hcmQiLCJNYW5hZ2UgVXNlcnMiLCJNYW5hZ2UgUm9sZXMiLCJWaWV3IEpvYnMiLCJWaWV3IExvZ2dpbmciLCJWaWV3IFNlcnZpY2VzIiwiRGV2ZWxvcGVyIiwiVmlldyBDdXN0b21lcnMiLCJBZGQgQ3VzdG9tZXJzIiwiTW9kaWZ5IEN1c3RvbWVycyIsIkltcG9ydCBDdXN0b21lcnMiLCJNYW5hZ2UgU0ZUUCBDcmVkZW50aWFscyIsIlZpZXcgQ3VzdG9tZXIgRGF0YSIsIlZpZXcgUHJvZHVjdCBEZXRhaWxzIiwiUHJvY2VzcyBGZWVkcyIsIlZpZXcgU2NvdXQgRmlsZSIsIk1vZGlmeSBQcm9kdWN0IERhdGEiLCJQdWJsaXNoIFByb2R1Y3QgRGF0YSIsIlJlY2VpdmUgVW5jdXJhdGVkIFByb2R1Y3QgRW1haWxzIiwiUmVjZWl2ZSBTdGFsZSBKb2IgRW1haWxzIiwiRGVsZXRlIEN1cmF0ZWQgRGF0YSIsIlNob3VsZCBOb3QgR2V0IERhc2hib2FyZCIsIk1pZ3JhdGUgQ3VyYXRpb24iLCJXaG9sZSBFbmNoaWxhZGEiLCJTdWJtaXQgUHJvZHVjdCBGZWVkYmFjayIsIlZpZXcgUmVwb3J0cyIsIlJ1biBSZXBvcnQiLCJHZW5lcmF0ZSBWaWV3cyIsIkdlbmVyYXRlIENhdGVnb3J5IFdoaXRlbGlzdCIsIlZpZXcgR2xvYmFsIFRheG9ub215IiwiRWRpdCBHbG9iYWwgVGF4b25vbXkiLCJWaWV3IEdsb2JhbCBDbGFzc2lmaWNhdGlvbiBSdWxlcyIsIkVkaXQgR2xvYmFsIENsYXNzaWZpY2F0aW9uIFJ1bGVzIiwiVmlldyBDdXN0b21lciBTdHJhdGVneSIsIkVkaXQgQ3VzdG9tZXIgU3RyYXRlZ3kiLCJFZGl0IEN1c3RvbWVyIFN0cmF0ZWd5IGluIFByZXZpZXciLCJBcHByb3ZlIEN1c3RvbWVyIFN0cmF0ZWd5IiwiRWRpdCBHdWlkZWxpbmVzIEluIEN1cmF0aW9uIiwiVmlldyBHdWlkZWxpbmVzIiwiRWRpdCBHdWlkZWxpbmVzIiwiVmlldyBDdXJhdGlvbiBKb2JzIiwiTWFuYWdlIEN1cmF0aW9uIEpvYnMiLCJFZGl0IEN1cmF0aW9uIFRhc2sgRmllbGRzIiwiUGVyZm9ybSBDdXJhdGlvbiBUYXNrcyIsIlBlcmZvcm0gUUEgQ3VyYXRpb24gVGFza3MiLCJQcmV2aWV3IEN1cmF0aW9uIFRhc2tzIiwiUmVjZWl2ZSBDdXJhdGlvbiBUYXNrIEVtYWlscyIsIlJlY2VpdmUgU3RyYXRlZ3kgQXBwcm92YWwgRW1haWxzIiwiQ2hhbmdlIFByb2R1Y3QgQnVja2V0IiwiTWFuYWdlIEN1cmF0aW9uIFJ1bGVzIiwiVmlldyBDdXJhdGlvbiBQZXJmb3JtYW5jZSIsIk1hbmFnZSBNYWNoaW5lIEN1cmF0aW9uIiwiVmlldyBJbnNpZ2h0cyIsIlZpZXcgVXNhZ2UgRGFzaGJvYXJkIiwiVmlldyBBdHRyaWJ1dGUgSW50ZWxsaWdlbmNlIl0sImlhdCI6MTU0OTQ4MTAyMiwiZXhwIjoxNTUwMDg1ODIyfQ.8Tfv5F2bOOJ-c2_GYshIWWSrkwO3cpxQimbTg0RhA9s"""

# Modular Functions

## Data from google sheets.

In [0]:
def get_worksheet_from_gsheets(URL,sheet_name):
    """
    Parameters:
    ------------
    URL: str url of the google sheets you would like to open.
    
    sheet_name: name of the worksheet inside the spreadsheet you are trying to open
    
    """
    gc = gspread.authorize(GoogleCredentials.get_application_default())
    spreadsheet = gc.open_by_url(URL)
    worksheet = spreadsheet.worksheet('Make Rules Out Of Me')
    df = pd.DataFrame(wks.get_all_records())
    return df

## Enrich data with Value IDs & Attribute IDs

In [0]:
def get_value_ids(df,search_product_type_column):
    """
    Description: Input the dataframe an the column name to get a dictionary of the value and the value_ids 
    ###########
    
    Parameters:
    ------------
    df: dataframe 
    
    column_name: str of the column name that contains the value names too look for
    
    """
    #Connect to SQL database
    conn = psycopg2.connect(**postgres)

    query = ("""
    SELECT
        id
        ,name
    FROM values
    """)

    values = pd.read_sql(query,conn)
    value_names = list(df[search_product_type_column].values)
    value_update_dict = {}
    #creating the dictionary from the google sheets workbook. 
    for name in value_names:
        try:
            value_update_dict.update({name:list(values[values['name'] == name]['id'])[0]})
        except:
            print('value name '+name+' name not in db')
            value_update_dict.update({name:0})
    #updating the data with values
    df['Value_ids'] = df[search_product_type_column].map(value_update_dict)
    return df

In [0]:
def get_attribute_ids(df,search_product_type_column):
    """
    Description: Input the dataframe an the column name to get a dictionary of the value and the value_ids 
    ###########
    
    Parameters:
    ------------
    df: dataframe 
    
    column_name: str of the column name that contains the value names too look for
    
    """
    #Connect to SQL database
    conn = psycopg2.connect(**postgres)

    query = ("""
    SELECT
      id  
      ,name  
    FROM attributes
    """)

    attri = pd.read_sql(query,conn)
    attri_names = list(df[search_product_type_column].values)
    attri_update_dict = {}
    #creating the dictionary from the google sheets workbook. 
    for name in attri_names:
        try:
            attri_update_dict.update({name:list(attri[attri['name'] == name]['id'])[0]})
        except:
            print('attribute_name '+name+' name not in db')
            attri_update_dict.update({name:0})
    #updating the data with values
    df['attribute_id'] = df[search_product_type_column].map(attri_update_dict)
    return df

## Generating the JSON file to load to the API
This runs for each row of data for the google sheets

In [0]:
def curation_rules(df,row,from_col,to_col):
    """
    Description: constructs the rules POST in JSON format
    ###########
    
    Parameters:
    -----------
    json_string: This is the POST body String to update
    
    value_to_update: This is a dictionary of {'value_name':'valud_id'}
    
    ------------
    
    
    """
    
    POST_body = """{
    "name": "Sandpaper",
    "conditions": [
        {
            "type": "FIELD_VALUE",
            "category_id": null,
            "field_name": "Q:product_type",
            "field_value": "Sandpaper",
            "value_comparison": "MATCHES"
        }
    ],
    "conditionsExpression": "A",
    "taxonomyVersionId": 1,
    "saveAsGlobal": false,
    "bucketId": null,
    "valueId": 27383,
    "type": "attribution",
    "curationRuleAttributes": [
        {
            "enabled": true,
            "customer_id": 55,
            "attribute_id": 4086
        }
    ]
        }"""
    body = json.loads(POST_body)
    #replace name with value nam 
    body.update({"name": "Bulk Rule Upload - {0} - {1}".format(df.iloc[row][to_col],today)})
    conditions = body["conditions"][0]
    conditions.update({"field_value":df.iloc[row][from_col]})
    body.update({"valueId":int(df.iloc[row]['Value_ids'])})
    body["curationRuleAttributes"][0].update({'customer_id':int(df.iloc[row]['Client_ID'])})
    body["curationRuleAttributes"][0].update({'attribute_id':int(df.iloc[row]['attribute_id'])})
    return body

## Generate the API to POST
This is setup to be generate by each row item but  can choose to send it to Staging or Production

In [0]:
def edgecase_api(customer_id,staging=True,JWT=JWT):
    """
    Parameters:
    -----------
    customer_id: should be in the 'Client ID' column. 
     
    staging: default == True
      chooses whether to post data to prod or stagin API
    
    JWT: this is your JWT you get come the console. 
    
    
    """
    customer_id = str(customer_id)
    #setting the params for posting to the site
    if staging:
        url = "https://console.edgecasestaging.com/api/customers/{}/curation_rules/".format(customer_id)
    else:
        url = "https://console.edgecaseprod.com/api/customers/{}/curation_rules/".format(customer_id)
    querystring = {"":""}
    headers = {
        'authorization': "Bearer "+JWT,
        'content-type': "application/json"
        }
    return url,querystring,headers

## POST data Via the API

In [0]:
def post_data(from_col='Curated Value',to_col='Curate As',staging=True,JWT=JWT):
  
    """
    Parameters:
    ------------
    from_col: This is the previously curated value
    
    to_col: This is the column witt the values to curate as. 
    
    staging: Which environment should this exist in? Default is True.
      Only switch to False if you are sure that it will be ready for production.
    
    JWT: This is your JWT as a string
    ------------------
    
    """
    
  
    
    df = get_worksheet_from_gsheets(gsheets_url,sheet_name)
    df = get_value_ids(df,to_col)
    df = get_attribute_ids(df,'To Be Curated Attribute')
    
    data_log= []
    for row in tqdm(range(df.shape[0])):
        body = curation_rules(df,row,from_col=from_col,to_col=to_col)
        customer_id= df.iloc[row]['Client_ID']
        
        edgecase_api_url, edgecase_api_querystring ,edgecase_api_headers = edgecase_api(customer_id,staging=staging,JWT=JWT)
        
        response = requests.request("POST", 
                                        edgecase_api_url, 
                                        data=json.dumps(body), 
                                        headers=edgecase_api_headers, 
                                        params=edgecase_api_querystring)
        data_log.append(response.text)
    post_data.data_log = data_log

# Setting up the data execution

In [0]:
#getting the Value IDs from the sandbox data
df = get_value_ids(df,'Curate As')
#getting the Attribute IDS from the sandbox data.
df = get_attribute_ids(df,'To Be Curated Attribute')

In [0]:
df

,Client_ID,Curate As,Curated Attribute,Curated Value,To Be Curated Attribute,Value_ids,attribute_id
0,21,Concrete,Product Type,Ascots,Search Product Type,6820,4086
1,21,Alabaster Glass,Product Type,Bow Ties,Search Product Type,21133,4086
2,21,Concrete,Product Type,Neck Ties,Search Product Type,6820,4086


In [0]:
#Post data to either the stating or products. 
post_data(staging=True,JWT=JWT)

100%|██████████| 3/3 [00:00<00:00,  9.96it/s]


In [0]:
#checking for invalid tokens
[data for data in post_data.data_log if data.find('invaild') > -1]

[]